In [ ]:
pip install scikit-dimension

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 1.8 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
from PIL import Image
from skdim.id import DANCo

In [ ]:
import os
import cv2
import numpy as np
from skdim.id import MLE
from sklearn.preprocessing import StandardScaler

# Definindo o caminho para a pasta de imagens
image_folder = '/content/drive/MyDrive/chest_xray/train'

# Lista para armazenar as imagens
images = []

# Carregando as imagens
for folder in os.listdir(image_folder):
    for file in os.listdir(os.path.join(image_folder, folder)):
        # Carregando a imagem em escala de cinza
        img = cv2.imread(os.path.join(image_folder, folder, file), cv2.IMREAD_GRAYSCALE)
        # Redimensionando a imagem para um tamanho fixo (por exemplo, 64x64)
        img = cv2.resize(img, (64, 64))
        # Achatar a imagem em um vetor unidimensional
        img = img.flatten()
        # Adicionando a imagem à lista
        images.append(img)

# Convertendo a lista de imagens em um array numpy
X = np.array(images)

# Normalizando os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Calculando a dimensão intrínseca
id_estimator = MLE()
id = id_estimator.fit_transform(X)

print('A dimensão intrínseca estimada é:', np.mean(id))


A dimensão intrínseca estimada é: 24.112243874537132


In [ ]:
dim_estimator = DANCo().fit(X)
local_dimension = dim_estimator.dimension_

print(f"A dimensão local estimada é: {local_dimension}")

A dimensão local estimada é: nan


In [ ]:
import os
import numpy as np
from sklearn.manifold import Isomap
from PIL import Image
import time

# Função para carregar imagens e convertê-las em vetores
def load_images(folder):
    images = []
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename)).convert('L') # Convertendo para escala de cinza
        img = img.resize((128, 128)) # Redimensionamento para um tamanho padrão
        img_array = np.array(img).flatten() # Achatando a imagem em um vetor
        images.append(img_array)
    return images

# Carregar imagens
healthy_lungs = load_images('/content/drive/MyDrive/chest_xray/train/NORMAL') # Caminho para imagens de pulmões saudáveis
sick_lungs = load_images('/content/drive/MyDrive/chest_xray/train/PNEUMONIA') # Caminho para imagens de pulmões doentes

# Combinar os dados
X = np.array(healthy_lungs + sick_lungs)

In [ ]:
# Aplicar Isomap
start_time = time.time()
embedding = Isomap(n_neighbors=50, n_components=30)
X_transformed = embedding.fit_transform(X)
end_time = time.time()

print(f"Isomap execution time: {end_time - start_time} seconds")
print(f"Number of components after Isomap: {X_transformed.shape[1]}")


Isomap execution time: 91.66718101501465 seconds
Number of components after Isomap: 30


In [ ]:
X.shape

(5223, 16384)

In [ ]:
X_transformed

array([[ 4415.51798528,  -833.23636618,  3948.68688556, ...,
         -300.35278365,   338.52544085,  -668.21410158],
       [ 3238.16041527,   185.6593215 ,  3850.47935659, ...,
         -979.15836554,  -545.53376226,   318.32764092],
       [ 4306.58190909,  3458.26096376,  -972.68044462, ...,
        -1312.09251498,  1572.32101681,   644.17633764],
       ...,
       [-3820.54487643,  1626.68618983, -5754.89051708, ...,
         -574.67466373,  -646.67653393,  -595.07661587],
       [-2791.02399917, -2669.75692663,  -404.25557641, ...,
         -751.85175165,   169.49485495,  1026.14895981],
       [  249.73566912, -2730.60109557, -3584.37666882, ...,
          262.00541378,  1291.17994814,   515.23499747]])

In [ ]:
from sklearn.manifold import Isomap
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr
import numpy as np

# Calculando as distâncias no conjunto de dados original
original_distances = squareform(pdist(X, 'euclidean'))

# Calculando as distâncias no conjunto de dados transformado
transformed_distances = squareform(pdist(X_transformed, 'euclidean'))

# Calculando o coeficiente de correlação de Spearman
correlation, _ = spearmanr(original_distances.flatten(), transformed_distances.flatten())

print(f"Correlação de Spearman entre as distâncias originais e transformadas: {correlation}")

Correlação de Spearman entre as distâncias originais e transformadas: 0.8963049950676522


In [ ]:
from sklearn.manifold import TSNE
import time

start_time = time.time()
X_embedded = TSNE(n_components=3, perplexity=50, learning_rate=200).fit_transform(X)
end_time = time.time()

print(f"t-SNE execution time: {end_time - start_time} seconds")
print(f"Number of components after t-SNE: {X_embedded.shape[1]}")


t-SNE execution time: 323.61122369766235 seconds
Number of components after t-SNE: 3


Limite maximo de componentes que esse metodo aceita é 3, é frequentemente usado para visualização e por isso comumente limitado a 2 ou 3 dimensões.


In [ ]:
# Calculando as distâncias no conjunto de dados original
original_distances = squareform(pdist(X, 'euclidean'))

# Calculando as distâncias no conjunto de dados transformado pelo t-SNE
transformed_distances_tsne = squareform(pdist(X_embedded, 'euclidean'))

# Calculando o coeficiente de correlação de Spearman
correlation_tsne, _ = spearmanr(original_distances.flatten(), transformed_distances_tsne.flatten())

print(f"Correlação de Spearman entre as distâncias originais e transformadas pelo t-SNE: {correlation_tsne}")

Correlação de Spearman entre as distâncias originais e transformadas pelo t-SNE: 0.6126573287088546


In [ ]:
from sklearn.manifold import LocallyLinearEmbedding
import time

start_time = time.time()
embedding = LocallyLinearEmbedding(n_neighbors=50, n_components=30)
X_transformed = embedding.fit_transform(X)
end_time = time.time()

print(f"LLE execution time: {end_time - start_time} seconds")
print(f"Number of components after LLE: {X_transformed.shape[1]}")

LLE execution time: 105.0662910938263 seconds
Number of components after LLE: 30


é importante notar que este método avalia especificamente a preservação das distâncias euclidianas, o que pode ou não refletir completamente a "informação" contida nos dados, especialmente para métodos não-lineares de redução de dimensionalidade como o LLE. O LLE foca em preservar relações locais lineares, então a correlação das distâncias euclidianas globais pode não capturar totalmente a eficácia do LLE.

In [ ]:
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr
import numpy as np

# Suponha que X é o seu conjunto de dados original e X_transformed é o resultado do LLE

# Calculando as distâncias no conjunto de dados original
original_distances = squareform(pdist(X, 'euclidean'))

# Calculando as distâncias no conjunto de dados transformado pelo LLE
transformed_distances_lle = squareform(pdist(X_transformed, 'euclidean'))

# Calculando o coeficiente de correlação de Spearman
correlation_lle, _ = spearmanr(original_distances.flatten(), transformed_distances_lle.flatten())

print(f"Correlação de Spearman entre as distâncias originais e transformadas pelo LLE: {correlation_lle}")


Correlação de Spearman entre as distâncias originais e transformadas pelo LLE: 0.5783952944726024


In [ ]:
!pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.2 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86831 sha256=ae3cd0d0d4a7774e79db88ef457e6d049c68ae04e8c1c867a45fa75e6392572a
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
Successfully built umap-learn


In [ ]:
import umap.umap_ as umap
import time

start_time = time.time()
reducer = umap.UMAP(n_neighbors = 30,  # Valor entre 15 a 50
min_dist = 0.3,   # Valor entre 0.1 a 0.5
n_components = 30,  # Geralmente 2 ou 3 para visualização
learning_rate = 100,   # Valor entre 1 a 200
spread = 1.0   )
X_transformed = reducer.fit_transform(X)
end_time = time.time()

print(f"UMAP execution time: {end_time - start_time} seconds")
print(f"Number of components after UMAP: {X_transformed.shape[1]}")

UMAP execution time: 128.1242949962616 seconds
Number of components after UMAP: 30


In [ ]:
from sklearn.manifold import Isomap
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr
import numpy as np

# Calculando as distâncias no conjunto de dados original
original_distances = squareform(pdist(X, 'euclidean'))

# Calculando as distâncias no conjunto de dados transformado
transformed_distances = squareform(pdist(X_transformed, 'euclidean'))

# Calculando o coeficiente de correlação de Spearman
correlation, _ = spearmanr(original_distances.flatten(), transformed_distances.flatten())

print(f"Correlação de Spearman entre as distâncias originais e transformadas: {correlation}")

Correlação de Spearman entre as distâncias originais e transformadas: 0.6298226995308446


In [ ]:
from sklearn.manifold import LocallyLinearEmbedding
import time

start_time = time.time()
embedding = LocallyLinearEmbedding(n_neighbors=496, n_components=30, method='hessian')
X_transformed = embedding.fit_transform(X)
end_time = time.time()

print(f"Hessian LLE execution time: {end_time - start_time} seconds")
print(f"Number of components after Hessian LLE: {X_transformed.shape[1]}")

Hessian LLE execution time: 2657.209179878235 seconds
Number of components after Hessian LLE: 30


In [ ]:
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr
import numpy as np

# Suponha que X é o seu conjunto de dados original e X_transformed é o resultado do LLE

# Calculando as distâncias no conjunto de dados original
original_distances = squareform(pdist(X, 'euclidean'))

# Calculando as distâncias no conjunto de dados transformado pelo LLE
transformed_distances_lle = squareform(pdist(X_transformed, 'euclidean'))

# Calculando o coeficiente de correlação de Spearman
correlation_lle, _ = spearmanr(original_distances.flatten(), transformed_distances_lle.flatten())

print(f"Correlação de Spearman entre as distâncias originais e transformadas pelo LLE Hessian: {correlation_lle}")

Correlação de Spearman entre as distâncias originais e transformadas pelo LLE Hessian: 0.7430580901205778
